Лабораторна робота №2

Для початку імпортуємо потрібні нам модулі:

In [ ]:
import urllib.request
import os
import matplotlib.pyplot as plt 
# %matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
import re

Створюємо функцію для завантаження даних з якими ми будемо працювати:

In [ ]:
def data_downloader(path):
    print("Importing")

    # Початок циклу завантаження для кожного регіону України
    for i in range(1, 28):
        url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=1981&year2=2024&type=Mean"
        vhi_url = urllib.request.urlopen(url)
        file_path = os.path.join(path, f'vhi_id_{i}.csv')
        with open(file_path, 'wb') as out:
            out.write(vhi_url.read())
        print(f"VHI is downloaded")
        return file_path

# data_downloader("")

Importing
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded
VHI is downloaded


Нормалізуємо щойно завантажені дані для роботи з ними:

In [ ]:
# Порожній масив використовується для створення спільного для всіх регіонів датафрейма
arr = []

# Задаємо назви нашим стовпчикам
headers = ['year', 'week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'provinceID']

# Масиви для перекладу індексів з англійського формату на український
provinces = ["Вінницька область", "Волинська область", "Дніпропетровська область", "Донецька область", "Житомирська область", "Закарпатська область", "Запорізька область", "Івано-Франківська область", "Київська область", "Кіровоградська область", "Луганська область", "Львівська область", "Миколаївська область", "Одеська область", "Полтавська область", "Рівенська область", "Сумська область", "Тернопільська область", "Харківська область", "Херсонська область", "Хмельницька область", "Черкаська область", "Чернівецька область", "Чернігівська область", "Республіка Крим", "м. Київ", "м. Севастополь"]
provinceID = [22,24,23,25,3,4,8,19,20,21,9,26,10,11,12,13,14,15,16,27,17,18,6,1,2,7,5]


IDs = dict(zip(range(1,28), provinceID))

for i in range(1,28):
    df = pd.read_csv(f'vhi_id_{i}.csv', index_col=False, header=1, names=headers)
    df['provinceID'] = i

    df["year"] = df["year"].apply(lambda x: re.sub('<[^<]+?>', '', x))
    df.rename(columns= {"VHI<br>": "VHI"})

    
    df = df.drop(df.loc[df['VHI'] == -1].index)
    df = df.iloc[:-1 , :]
    df = df.astype({"year": "int64", "week": "int64"})
    arr.append(df)


concatDf = pd.concat(arr)
concatDf['provinceID'] = concatDf['provinceID'].replace(IDs)
concatDf = concatDf.sort_values(["provinceID", "year", "week"])

1. Процедура аналізу значень VHI для заданого датафрейму:

In [ ]:
def VHI_analysis(df):
    weekNum = df.shape[0]
    df_good = df[(df["VHI"] > 60)].shape[0] / weekNum
    df_medium = df[(df["VHI"] <= 60) & (df["VHI"] > 40)].shape[0] / weekNum
    df_stress = df[(df["VHI"] <= 40) & (df["VHI"] > 15)].shape[0] / weekNum
    df_drought = df[(df["VHI"] <= 15)].shape[0] / weekNum
    print(f"{2 * '    '}Відсоток тижнів з сприятливими умовами: {round(df_good, 3)}")
    print(f"{2 * '    '}Відсоток тижнів з достатніми умовами: {round(df_medium, 3)}")
    print(f"{2 * '    '}Відсоток тижнів з стресовими умовами: {round(df_stress, 3)}")
    print(f"{2 * '    '}Відсоток тижнів з посухою: {round(df_drought, 3)}") 



Процедура, яка виводить відсоток тижнів за змістом VHI-індексу для заданих регіонів за задані роки:

In [ ]:
def yearly_VHI(df = concatDf, names = provinces, years = None, index = None):

    # Процес введення даних для розрахунку VHI
    if years == None: 
        years = input("Введіть рік(-и) для виведення інформації: ")
        years = list(map(int, years.split()))
    if index == None:
        index = int(input("Введіть індекс регіону (1-27): "))

    print(f"Для {names[index-1]}: ")

    # Цикл аналізу введених даних
    for i in range(len(years)):
        yearlyDf = df[(df["provinceID"] == index) & (df["year"] == years[i])]
        printDf = yearlyDf[["VHI","week"]]
        print(f"    За {years[i]} рік:")
        VHI_analysis(yearlyDf)
        print(printDf)
    print(40 * "~")

#yearly_VHI()

Для Львівська область: 
    За 2020 рік:
        Відсоток тижнів з сприятливими умовами: 0.096
        Відсоток тижнів з достатніми умовами: 0.904
        Відсоток тижнів з стресовими умовами: 0.0
        Відсоток тижнів з посухою: 0.0
        VHI  week
1976  45.02     1
1977  45.81     2
1978  46.16     3
1979  46.20     4
1980  46.67     5
1981  47.14     6
1982  48.05     7
1983  49.84     8
1984  51.32     9
1985  52.14    10
1986  53.16    11
1987  53.81    12
1988  51.27    13
1989  47.56    14
1990  45.78    15
1991  47.34    16
1992  49.79    17
1993  51.13    18
1994  54.39    19
1995  57.88    20
1996  59.90    21
1997  60.95    22
1998  60.48    23
1999  59.74    24
2000  59.01    25
2001  58.02    26
2002  58.64    27
2003  59.93    28
2004  61.67    29
2005  63.06    30
2006  60.22    31
2007  56.67    32
2008  54.24    33
2009  53.99    34
2010  53.07    35
2011  50.38    36
2012  47.51    37
2013  44.83    38
2014  43.52    39
2015  44.85    40
2016  42.51    41
2017  42

2. Процедура аналізу максимального, мінімального, середнього та медіани VHI за задані роки в заданих регіонах:

In [ ]:
def important_data(df = concatDf, names = provinces):
    # Процес введення даних для аналізу VHI
    years = input("Введіть рік(-и) для виведення інформації: ")
    years = list(map(int, years.split()))
    indexes = input("Введіть індекс(-и) регіону(-ів) (1-27): ")
    indexes = list(map(int, indexes.split()))

    # Цикл аналізу
    for i in range(len(indexes)):
        print(f"Для {indexes[i]} регіону ({names[indexes[i]-1]}):")
        for j in range(len(years)):
            yearlyDf = df[(df["provinceID"] == indexes[i]) & (df["year"] == years[j])]["VHI"]
            min_v = yearlyDf.min()
            max_v = yearlyDf.max()
            avg_v = yearlyDf.mean()
            median_v = yearlyDf.median()
            print(f"    За {years[j]} рік:")
            print(f"{2 * '    '}Максимальне значення VHI: {max_v}")
            print(f"{2 * '    '}Мінімальне значення VHI: {min_v}")
            print(f"{2 * '    '}Середнє значення VHI: {round(avg_v, 3)}")
            print(f"{2 * '    '}Медіана VHI: {round(median_v, 3)}")
        print(40 * "~")

#important_data()

Для 16 регіону (Рівенська область):
    За 2020 рік:
        Максимальне значення VHI: 66.01
        Мінімальне значення VHI: 37.68
        Середнє значення VHI: 50.839
        Медіана VHI: 49.665
    За 1998 рік:
        Максимальне значення VHI: 74.2
        Мінімальне значення VHI: 40.68
        Середнє значення VHI: 57.483
        Медіана VHI: 57.525
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Для 21 регіону (Хмельницька область):
    За 2020 рік:
        Максимальне значення VHI: 67.65
        Мінімальне значення VHI: 37.43
        Середнє значення VHI: 49.138
        Медіана VHI: 45.615
    За 1998 рік:
        Максимальне значення VHI: 70.52
        Мінімальне значення VHI: 42.85
        Середнє значення VHI: 56.228
        Медіана VHI: 57.925
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


3. Процедура що виконує ту ж саму функцію що процедура 1 тільки для діапазону років:

In [ ]:
def VHI_range(df = concatDf, names = provinces):
    years = input("Введіть діапазон років для виведення інформації: ")
    start_and_end_years = list(map(int, years.split()))
    years = [j for j in range(start_and_end_years[0], start_and_end_years[1]+1)]
    indexes = input("Введіть індекс(-и) регіону(-ів) (1-27): ")
    indexes = list(map(int, indexes.split()))
    for i in indexes:
        yearly_VHI(years=years, index=i)

#VHI_range()

Для Херсонська область: 
    За 2020 рік:
        Відсоток тижнів з сприятливими умовами: 0.058
        Відсоток тижнів з достатніми умовами: 0.615
        Відсоток тижнів з стресовими умовами: 0.327
        Відсоток тижнів з посухою: 0.0
        VHI  week
1976  45.94     1
1977  45.28     2
1978  44.67     3
1979  44.62     4
1980  44.56     5
1981  44.91     6
1982  44.90     7
1983  45.07     8
1984  45.25     9
1985  45.00    10
1986  44.59    11
1987  45.10    12
1988  46.75    13
1989  48.09    14
1990  49.77    15
1991  52.60    16
1992  55.69    17
1993  57.93    18
1994  61.80    19
1995  63.49    20
1996  62.18    21
1997  59.98    22
1998  57.35    23
1999  53.68    24
2000  50.56    25
2001  47.67    26
2002  45.29    27
2003  43.91    28
2004  42.69    29
2005  41.22    30
2006  39.86    31
2007  37.98    32
2008  35.40    33
2009  32.74    34
2010  29.58    35
2011  25.81    36
2012  24.49    37
2013  25.05    38
2014  27.68    39
2015  32.59    40
2016  35.98    41
2017 

4. Процедура яка аналізує весь датафрейм і повертає середнье значення VHI за кожен рік для кожного регіону та % регіонів яких торкнулися екстремальні посухи:

In [36]:
def VHI_Summarizer(df):
    VHI_data = dict()
    VHI_pers = dict()
    for i in range(1982, 2025):
        arr = []
        count = 0
        for j in range(1, 28): 
            yearlyDf = df[(df["provinceID"] == j) & (df["year"] == i)]["VHI"]
            avg_v =  yearlyDf.mean()
            avg_v
            if avg_v < 35:
                count += 1
            arr.append(round(avg_v, 3))
        VHI_data.update({i: arr})
        VHI_pers.update({i: count/28*100})
    return VHI_data, VHI_pers

Процедура яка приймає відсоток регіонів і використовує попереднью процедуру для визначення років з екстремальними посухами та регіони що відчули ці посухи: 

In [ ]:
def droughtPersentage(df = concatDf, provinces = provinces):
    pers = int(input("Введіть відсоток для пошуку: "))
    VHI_data, VHI_pers = VHI_Summarizer(df)
    for key, i in VHI_pers.items():
        if pers < i:
            year_data = VHI_data[key]
            print(f"Екстремальні посухи торкнулися більше {pers} відсотків проівнцій протягом {key} року")
            for j in year_data:
                if j < 35:
                    print(f"    {provinces[year_data.index(j)]} зі значенням VHI - {j}")
            print(40*"~")


#droughtPersentage()

Екстремальні посухи торкнулися більше 1 відсотків проівнцій протягом 1986 року
    Вінницька область зі значенням VHI - 34.906
    Дніпропетровська область зі значенням VHI - 30.619
    Донецька область зі значенням VHI - 27.871
    Запорізька область зі значенням VHI - 31.08
    Кіровоградська область зі значенням VHI - 30.35
    Луганська область зі значенням VHI - 26.813
    Миколаївська область зі значенням VHI - 31.969
    Одеська область зі значенням VHI - 33.025
    Полтавська область зі значенням VHI - 33.604
    Харківська область зі значенням VHI - 31.588
    Черкаська область зі значенням VHI - 34.355
    Республіка Крим зі значенням VHI - 34.858
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Екстремальні посухи торкнулися більше 1 відсотків проівнцій протягом 1989 року
    м. Севастополь зі значенням VHI - 32.105
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Екстремальні посухи торкнулися більше 1 відсотків проівнцій протягом 1990 року
    м. Севастополь зі значенням VHI - 29.88
~~~~~